<a href="https://colab.research.google.com/github/hyeong01/hyeong01.github.io/blob/master/example_code_stoarge/Saving_a_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import os

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [28]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(16,32,kernel_size=3,stride=2,padding=0),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2))
        
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(3*3*64, 1000)
        self.fc2 = nn.Linear(1000, 1)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)

        out = out.view(out.size(0),-1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out


In [29]:
model = Model()
if torch.cuda.is_available():
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr = 0.1, momentum=0.9)


In [30]:
print("Model state:")
for param in model.state_dict():
    print(param, "   ", model.state_dict()[param].size())

Model state:
layer1.0.weight     torch.Size([16, 3, 3, 3])
layer1.0.bias     torch.Size([16])
layer1.1.weight     torch.Size([16])
layer1.1.bias     torch.Size([16])
layer1.1.running_mean     torch.Size([16])
layer1.1.running_var     torch.Size([16])
layer1.1.num_batches_tracked     torch.Size([])
layer2.0.weight     torch.Size([32, 16, 3, 3])
layer2.0.bias     torch.Size([32])
layer2.1.weight     torch.Size([32])
layer2.1.bias     torch.Size([32])
layer2.1.running_mean     torch.Size([32])
layer2.1.running_var     torch.Size([32])
layer2.1.num_batches_tracked     torch.Size([])
layer3.0.weight     torch.Size([64, 32, 3, 3])
layer3.0.bias     torch.Size([64])
layer3.1.weight     torch.Size([64])
layer3.1.bias     torch.Size([64])
layer3.1.running_mean     torch.Size([64])
layer3.1.running_var     torch.Size([64])
layer3.1.num_batches_tracked     torch.Size([])
fc1.weight     torch.Size([1000, 576])
fc1.bias     torch.Size([1000])
fc2.weight     torch.Size([1, 1000])
fc2.bias     torch.

In [27]:
from torchsummary import summary
summary(model, (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 111, 111]             448
       BatchNorm2d-2         [-1, 16, 111, 111]              32
              ReLU-3         [-1, 16, 111, 111]               0
         MaxPool2d-4           [-1, 16, 55, 55]               0
            Conv2d-5           [-1, 32, 27, 27]           4,640
       BatchNorm2d-6           [-1, 32, 27, 27]              64
              ReLU-7           [-1, 32, 27, 27]               0
         MaxPool2d-8           [-1, 32, 13, 13]               0
            Conv2d-9             [-1, 64, 6, 6]          18,496
      BatchNorm2d-10             [-1, 64, 6, 6]             128
             ReLU-11             [-1, 64, 6, 6]               0
        MaxPool2d-12             [-1, 64, 3, 3]               0
          Dropout-13                  [-1, 576]               0
           Linear-14                 [-

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [32]:
# save parameter
MODEL_PATH = "saved_models"
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)
torch.save(model.state_dict(), os.path.join(MODEL_PATH, "model.pt"))

In [33]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
another_model = Model()
another_model.load_state_dict(torch.load(os.path.join(MODEL_PATH, 'model.pt')))

<All keys matched successfully>

In [34]:
# save whole model
torch.save(model, os.path.join(MODEL_PATH, "whole_model.pt"))
model = torch.load(os.path.join(MODEL_PATH, "whole_model.pt"))
model.eval()

Model(
  (layer1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (drop_out): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=576, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=1, bias=True